I will perform basic sentiment analysis on collected COVID19-related Twitter data, using the TextBlob entiment analysis tool.
This tool uses a lexicon (i.e. a dictionary/vocabulary of words and their respective sentiment scores) and rule-based approach to classify text as either negative, neutral or positive.
However, the fact that sentiment is calculated based on the polarity score of each word in the text can lead to errors in sentiment classification. Such errors may arise due to a lack of understanding of various language traits such as sarcasm or even variations in the use of word the same word. 

In [1]:

import pandas as pd
import csv
import re
import numpy as np
import plotly.express as px
from plotly.offline import init_notebook_mode
!pip install vaderSentiment

from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 6.0 MB/s 


We now read in our Twitter data that has been partially cleaned and put it into a pandas dataframe. 

In [3]:
# Read in the data and put it into a dataframe
df = pd.read_csv("/content/dataframe_csv.csv")

# Have a quick look at the dataframe
df

,Unnamed: 0,tweet_id_str,date_time,location,tweet_text
0,0,1529612798241959943,2022-05-25 23:58:04,Kenya,Water melon for Ksh18.00 @ kg in Bura tana. Dm...
1,1,1529611602487611393,2022-05-25 23:53:19,Kenya,Dennis Itumbi abduction - Angry Rigathi Gachag...
2,2,1529611298098618368,2022-05-25 23:52:06,Ethiopia,The Amhara regional state government has engag...
3,3,1529610866014007296,2022-05-25 23:50:23,Kenya,LIVE | DP William Ruto in Mombasa https://t.co...
4,4,1529610794933305345,2022-05-25 23:50:06,Kenya,"pigs for Ksh6,000.00 @ piglet in Nakuru. high ..."
...,...,...,...,...,...
995,995,1529447378809503744,2022-05-25 13:00:45,Kenya,DP Ruto picks Mathira MP Rigathi Gachagua as h...
996,996,1529447328318464001,2022-05-25 13:00:33,Tanzania,DO YOU LIKE DISCOUNTS? Get more for less by us...
997,997,1529447261117239302,2022-05-25 13:00:17,"Nairobi, Kenya","Launching in June 2022, the #AfricanFutures ca..."
998,998,1529447243270475777,2022-05-25 13:00:13,Kitui,"Raila Should ""Boss"" About Coast Land Grabbing ..."


In [4]:
#Create a function to get the subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

#Create a function to get the polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

Let's try out these TextBlob tools by entering any text you'd like - try different examples, i.e. text which you think has positive or negative sentiment and see if TextBlob can get it right! Does punctuation make a difference? How does it handle sarcasm or common text slang such as 'lol'?

In [5]:
your_text = 'this is so cool'
getPolarity(your_text), getSubjectivity(your_text)

(0.35, 0.65)

In [6]:

df['subjectivity'] = df['tweet_text'].apply(getSubjectivity)
df['polarity'] = df['tweet_text'].apply(getPolarity)

df

,Unnamed: 0,tweet_id_str,date_time,location,tweet_text,subjectivity,polarity
0,0,1529612798241959943,2022-05-25 23:58:04,Kenya,Water melon for Ksh18.00 @ kg in Bura tana. Dm...,0.500000,0.500000
1,1,1529611602487611393,2022-05-25 23:53:19,Kenya,Dennis Itumbi abduction - Angry Rigathi Gachag...,1.000000,-0.625000
2,2,1529611298098618368,2022-05-25 23:52:06,Ethiopia,The Amhara regional state government has engag...,0.000000,0.000000
3,3,1529610866014007296,2022-05-25 23:50:23,Kenya,LIVE | DP William Ruto in Mombasa https://t.co...,0.500000,0.136364
4,4,1529610794933305345,2022-05-25 23:50:06,Kenya,"pigs for Ksh6,000.00 @ piglet in Nakuru. high ...",0.520000,0.330000
...,...,...,...,...,...,...,...
995,995,1529447378809503744,2022-05-25 13:00:45,Kenya,DP Ruto picks Mathira MP Rigathi Gachagua as h...,0.000000,0.000000
996,996,1529447328318464001,2022-05-25 13:00:33,Tanzania,DO YOU LIKE DISCOUNTS? Get more for less by us...,0.266667,0.066667
997,997,1529447261117239302,2022-05-25 13:00:17,"Nairobi, Kenya","Launching in June 2022, the #AfricanFutures ca...",0.250000,0.125000
998,998,1529447243270475777,2022-05-25 13:00:13,Kitui,"Raila Should ""Boss"" About Coast Land Grabbing ...",0.000000,0.000000


Next, let's create a function to add a sentiment label to each tweet, based on it's polarity score.

In [7]:
# Create a function to label postitive, neutral and negative tweets

def get_sentiment_label(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'  

In [8]:
# Apply the get_sentiment_label function to the polarity column
# and add the sentiment results as a new column in our dataframe

df['TBsentiment'] = df['polarity'].apply(get_sentiment_label)
df

,Unnamed: 0,tweet_id_str,date_time,location,tweet_text,subjectivity,polarity,TBsentiment
0,0,1529612798241959943,2022-05-25 23:58:04,Kenya,Water melon for Ksh18.00 @ kg in Bura tana. Dm...,0.500000,0.500000,Positive
1,1,1529611602487611393,2022-05-25 23:53:19,Kenya,Dennis Itumbi abduction - Angry Rigathi Gachag...,1.000000,-0.625000,Negative
2,2,1529611298098618368,2022-05-25 23:52:06,Ethiopia,The Amhara regional state government has engag...,0.000000,0.000000,Neutral
3,3,1529610866014007296,2022-05-25 23:50:23,Kenya,LIVE | DP William Ruto in Mombasa https://t.co...,0.500000,0.136364,Positive
4,4,1529610794933305345,2022-05-25 23:50:06,Kenya,"pigs for Ksh6,000.00 @ piglet in Nakuru. high ...",0.520000,0.330000,Positive
...,...,...,...,...,...,...,...,...
995,995,1529447378809503744,2022-05-25 13:00:45,Kenya,DP Ruto picks Mathira MP Rigathi Gachagua as h...,0.000000,0.000000,Neutral
996,996,1529447328318464001,2022-05-25 13:00:33,Tanzania,DO YOU LIKE DISCOUNTS? Get more for less by us...,0.266667,0.066667,Positive
997,997,1529447261117239302,2022-05-25 13:00:17,"Nairobi, Kenya","Launching in June 2022, the #AfricanFutures ca...",0.250000,0.125000,Positive
998,998,1529447243270475777,2022-05-25 13:00:13,Kitui,"Raila Should ""Boss"" About Coast Land Grabbing ...",0.000000,0.000000,Neutral


We can have a quick look at the sentiment distribution of the tweets as follows:

In [9]:
df['TBsentiment'].value_counts()

Neutral     429
Positive    375
Negative    196
Name: TBsentiment, dtype: int64

Let's have a closer look at the tweets which TexBlob has classified as the most positive and most negative, to check the accuracy of the assigned sentiment. To do this, we will first sort all the tweets by polarity in descending order, i.e. from the most positive tweets to the most negative.

In [10]:
#We sort the tweets by their polarity value and put the sorted tweets into a new dataframe 

sorted_df = df.sort_values(by=['polarity'], ascending=False)

Next, let's print out the top 15 most negative tweets, which are now the last 15 tweets in the new dataframe. Do these tweets have a negative sentiment?

In [11]:
#Print out the text from the last 15 tweets in the sorted dataframe

for i, tweet in enumerate(sorted_df.tail(15)['tweet_text']):
    print(i+1, tweet, '\n')

1 Dear William Ruto , this is a woman , a leader , a gubernatorial candidate by the name Aisha Jumwa vying under UDA  you’re frustrating her because of Amason Kingi who joined your camp two weeks ago , why is she begging to speak at your rallies ? Is this how your reward your foot! https://t.co/HDCrgzcz0b 

2 Chicken for Ksh900.00 @ per hen in Sagana. Clearance Sale, Farm is located along Nyeri-Nairobi Highway
Order : https://t.co/0l1PuxIhap
Use App : https://t.co/L7r2Sz4eHl
AD: https://t.co/DdPensGWLk https://t.co/DXAWgBuI48 

3 chicken for Ksh1,200.00 @  in nairobi. 
Order : https://t.co/pZk68FLtnD
Use App : https://t.co/L7r2Sz4eHl
Express Way | https://t.co/DdPensGWLk https://t.co/ggLK8vWtkU 

4 Kienyenji Chicken for Ksh500.00 @ Piece in Thika. John
Order : https://t.co/w6UFu1QxZW
Use App : https://t.co/L7r2Sz4eHl https://t.co/QlOx7HbTqC 

5 What's happening in the yUDA camp?
1. Wamuchomba throws food remains to a crowd
2. Kimani Ichungwa calls HUSTLERS dirty people
3. Aisha Jumwa c

In [12]:
init_notebook_mode(connected=True)
